# NBA Data Analysis :: API Connection Part 2

## Trevor Rowland :: 2-5-2025

This notebook aims to collect all teams that made the playoffs in the 2004-2024 seasons, and also a Play-by-Play data grabber to use for later projects.

## 1. Collecting Playoff Teams

This function uses the PlayoffPicture endpoint from `nba_api` to get a glimpse of the playoffs in each season. This glimpse contains all of the teams who made the playoffs, which will be used in our 4th hypothesis test using MANOVA to compare playoff teams to non-playoff-making teams.

**Edit:** We are no longer using the `nba_api` endpoints because that is stupid overengineering. Instead, we can just write down every playoff team from wikipedia instead of wrestling with the API.

**Edit 2:** Even Better! All `game_id` values contain whether the game is a playoff game or not.

Playoff Games will start off with the numbers 00...

- 1: Pre-Season Games
- 2: Regular Season Games
- 3: All-Star Games
- 4: Playoff Games

**[Source](<https://github.com/swar/nba_api/issues/220>)**

This means that we are still waiting to get the game data but at least we have an elegant solution when we get those `game_id` datasets.

### 1.a. Playoff Teams

The following is a dictionary of each playoff team that made the NBA playoffs since 2004.

In [ ]:
import pandas as pd
import numpy as np

### 1.a. Importing Packages